In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import corner
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_probability as tfp

tfk = tf.keras
tfd = tfp.distributions
tfb = tfp.bijectors

seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
def timer(func):
    
    t0 = time.time()
    x = func()
    print(time.time() - t0)
    
    return x

In [ ]:
dims = 2

In [ ]:
distribution = tfd.Sample(
    tfd.Normal(loc=0., scale=1.),
    sample_shape=[dims],
    )

bijector = tfb.MaskedAutoregressiveFlow(
    tfb.AutoregressiveNetwork(
        params=2,
        hidden_units=[512, 512],
        ),
    )

In [ ]:
maf = tfd.TransformedDistribution(
    distribution=distribution,
    bijector=bijector,
    )

In [ ]:
sample = timer(lambda: maf.sample(10000))
lp = timer(lambda: maf.log_prob(sample))

In [ ]:
iaf = tfd.TransformedDistribution(
    distribution=distribution,
    bijector=tfb.Invert(bijector),
    )

In [ ]:
sample = timer(lambda: iaf.sample(10000))
lp = timer(lambda: maf.log_prob(sample))

In [ ]:
distribution = tfd.Sample(
    tfd.Normal(loc=0., scale=1.),
    sample_shape=[dims],
    )

bijectors = []
for _ in range(2):
    made = tfb.AutoregressiveNetwork(params=2, hidden_units=[1024])
    bijector = tfb.MaskedAutoregressiveFlow(made)
    bijectors.append(bijector)
bijector = tfb.Chain(bijectors)

maf = tfd.TransformedDistribution(distribution=distribution, bijector=bijector)
iaf = tfd.TransformedDistribution(distribution=distribution, bijector=tfb.Invert(bijector))

In [ ]:
sample = timer(lambda: maf.sample(10000))
lp = timer(lambda: maf.log_prob(sample))
corner.corner(sample.numpy())
plt.show()

sample = timer(lambda: iaf.sample(10000))
lp = timer(lambda: iaf.log_prob(sample))
corner.corner(sample.numpy())
plt.show()

In [ ]:
# https://github.com/tensorflow/probability/issues/1410
# https://github.com/tensorflow/probability/issues/1006#issuecomment-663141106

def make_kwargs(flow, condition):
    
    prefix = flow.bijector.bijectors[-1].name[:-1]
    
    return {
        b.name: {'conditional_input': condition}
        for b in flow.bijector.bijectors if prefix in b.name
        }

In [ ]:
dims = 2
cond_dims = 1

In [ ]:
distribution = tfd.Sample(
    tfd.Normal(loc=0., scale=1.),
    sample_shape=[dims],
    )

bijectors = []
for _ in range(10):
    made = tfb.AutoregressiveNetwork(
        params=2,
        event_shape=(dims,),
        conditional=True,
        conditional_event_shape=(cond_dims,),
        hidden_units=[1024],
        )
    bijector = tfb.MaskedAutoregressiveFlow(made, name=f'maf{_}')
    bijectors.append(bijector)
bijector = tfb.Chain(bijectors)

maf = tfd.TransformedDistribution(distribution=distribution, bijector=bijector)
iaf = tfd.TransformedDistribution(distribution=distribution, bijector=tfb.Invert(bijector))

In [ ]:
kw = make_kwargs(maf, [1.])

sample = timer(lambda: maf.sample(10000, bijector_kwargs=kw))
lp = timer(lambda: maf.log_prob(sample, bijector_kwargs=kw))
corner.corner(sample.numpy())
plt.show()

sample = timer(lambda: iaf.sample(10000, bijector_kwargs=kw))
lp = timer(lambda: iaf.log_prob(sample, bijector_kwargs=kw))
corner.corner(sample.numpy())
plt.show()